In [9]:
import app
import importlib
importlib.reload(app)
from app import (
    build_model_and_transforms,
    get_device,
    get_args_parser,
    generate_heatmap,
    get_xy_from_boxes,
    predict,
)
import importlib
import logging
from pathlib import Path
from PIL import Image
import json
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import time

# Get the argument parser and update the default for pretrain_model_path
parser = get_args_parser()
parser.set_defaults(pretrain_model_path="checkpoint_best_regular.pth")
args = parser.parse_args([])

device = get_device()
model1, transform = build_model_and_transforms(args)
model1 = model1.to(device)

# Define a lambda function to run predictions
run1 = lambda image, text: predict(model1, transform, image, text, None, device)

# Define a lambda function to process output given an image and boxes
get_output = lambda image, boxes: (len(boxes), get_xy_from_boxes(image, boxes), generate_heatmap(image, boxes))
# Only output the count and the coordinates, without generating the heatmap image.
get_output_count = lambda image, boxes: (len(boxes), get_xy_from_boxes(image, boxes))

# Now, args.pretrain_model_path will be "checkpoint_best_regular.pth" unless overridden.
print("Pretrained model path:", args.pretrain_model_path)


Some weights of BertModel were not initialized from the model checkpoint at checkpoints/bert-base-uncased and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


final text_encoder_type: checkpoints/bert-base-uncased
load tokenizer done.
final text_encoder_type: checkpoints/bert-base-uncased
load tokenizer done.
Pretrained model path: checkpoint_best_regular.pth


In [13]:
parser = get_args_parser()
parser.set_defaults(pretrain_model_path=r"/mnt/sda1/PythonProject/Pig_counting/Pig_farming/train_countgd/checkpoints/papercheckpoint_thinh_train_199.pth")
args = parser.parse_args([])

# Load the first model (e.g., using the default path)
device = get_device()
model2, transform1 = build_model_and_transforms(args)
model2 = model2.to(device)

run2 = lambda image, text: predict(model2, transform, image, text, None, device)
print("Pretrained model path:", args.pretrain_model_path)


Some weights of BertModel were not initialized from the model checkpoint at checkpoints/bert-base-uncased and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


final text_encoder_type: checkpoints/bert-base-uncased
load tokenizer done.
final text_encoder_type: checkpoints/bert-base-uncased
load tokenizer done.
Pretrained model path: /mnt/sda1/PythonProject/Pig_counting/Pig_farming/train_countgd/checkpoints/papercheckpoint_thinh_train_199.pth


In [14]:


# Import the utility function to convert boxes to centers

# Setup logging
logger = logging.getLogger()
logger.setLevel(logging.INFO)

# Define model names (for display purposes only)
model1_name = "model_base"
model2_name = "papercheckpoint_thinh_train_199"

def process_folder_with_two_models(image_path: Path, text: str, output_dir: Path):
    """Processes an image using two models, logs results, and saves a comparison heatmap."""
    total_time = time.time()
    # Load the original image
    try:
        with Image.open(image_path) as im:
            im.load()
            original_image = im.convert("RGB")
    except Exception as e:
        logger.error(f"Error loading image {image_path}: {e}")
        return

    # --- Run model 1 ---
    start_time_1 = time.time()
    boxes1, _ = run1(original_image, text)  # Ensure run1 is defined
    count1, xy1=get_output_count(original_image, boxes1)
    latency_1 = time.time() - start_time_1
    fps_1 = 1 / latency_1 if latency_1 > 0 else float('inf')

    # --- Run model 2 ---
    start_time_2 = time.time()
    boxes2, _ = run2(original_image, text)
    count2, xy2=get_output_count(original_image, boxes2)
    latency_2 = time.time() - start_time_2
    fps_2 = 1 / latency_2 if latency_2 > 0 else float('inf')

    # Get outputs from both models
    count1, xy1, heatmap1 = get_output(original_image, boxes1)
    count2, xy2, heatmap2 = get_output(original_image, boxes2)
    end_time = time.time()
    total_latency = end_time - total_time
    total_fps = 1 / total_latency if total_latency > 0 else float('inf')
    # Log predictions and performance metrics
    logger.info(f"{model1_name} predicts {count1}, Latency: {latency_1:.2f} sec, FPS: {fps_1:.2f}")
    logger.info(f"{model2_name} predicts {count2}, Latency: {latency_2:.2f} sec, FPS: {fps_2:.2f}")

    print("Predict count 1:",count1,"\n"
          ,"Predict count 2:",count2)
    # Ensure heatmaps have matching height before concatenating

    heatmap_combined = np.concatenate([np.array(heatmap1), np.array(heatmap2)], axis=1)

    # Create a figure with two subplots
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(24, 12), dpi=100)

    # --- Left subplot: original image with ground truth points ---
    ax1.imshow(np.array(original_image))
    ax1.set_title("Original Image with Ground Truth", fontsize=20)
    ax1.axis('off')

    # --- Right subplot: Combined model prediction heatmap with counts ---
    ax2.imshow(heatmap_combined, cmap='jet')
    ax2.set_title("Model Predictions", fontsize=20)

    # Overlay prediction text for each model
    ax2.text(20, 40, f"{model1_name}: {count1}", fontsize=18, color="white", backgroundcolor="black")
    ax2.text(heatmap_combined.shape[1] // 2 + 20, 40, f"{model2_name}: {count2}", fontsize=18, color="white", backgroundcolor="black")

    # Add latency and FPS info to the heatmap
    latency_text = (f"{model1_name}: Latency {latency_1:.2f} sec | FPS {fps_1:.2f}\n"
                    f"{model2_name}: Latency {latency_2:.2f} sec | FPS {fps_2:.2f}\n"
                    f"Total Latency: {total_latency:.2f} sec | Total FPS: {total_fps:.2f}")
    ax2.text(20, heatmap_combined.shape[0] + 80, latency_text, fontsize=18, color="white", backgroundcolor="black")

    # Save the combined results
    plt.tight_layout()
    output_plot_path = output_dir / f"{image_path.stem}_combined_results.png"
    plt.savefig(output_plot_path, bbox_inches='tight', pad_inches=0)
    plt.close(fig)

    logger.info(f"Combined plot saved at {output_plot_path}")

    return count1, count2, xy1, xy2, heatmap_combined



In [15]:
import os
import glob
from pathlib import Path

# Define the input folder containing test images (e.g., "img200")
input_folder = r"/mnt/sda1/PythonProject/Pig_counting/dataset/mini_pig_200img"

# Define the output directory (this should match what you use in your processing function)
output_dir = Path(r"/mnt/sda1/PythonProject/Pig_counting/dataset/mini_pig_200img/papercheckpoint_thinh_train_199")
output_dir.mkdir(parents=True, exist_ok=True)

# Get all image files (recursively) in the input folder
image_files = glob.glob(os.path.join(input_folder, "**/*.jpg"), recursive=True)

object_to_count = "pig"  # the object label you're interested in

# Iterate over each image file and process it
for img_file in image_files:
    print(f"Processing {img_file}...")
    process_folder_with_two_models(Path(img_file), object_to_count,output_dir)


Processing /mnt/sda1/PythonProject/Pig_counting/dataset/mini_pig_200img/2023-04-12-09-50-03_2023-04-12-10-05-03_8851_jpg.rf.e1bf24c253e9dcdfb3bba15d6a3b4922.jpg...
Predict count 1: 58 
 Predict count 2: 50
Processing /mnt/sda1/PythonProject/Pig_counting/dataset/mini_pig_200img/1472_jpg.rf.12b397be7817e49e9e740dea8c537715.jpg...
Predict count 1: 23 
 Predict count 2: 25
Processing /mnt/sda1/PythonProject/Pig_counting/dataset/mini_pig_200img/1403_jpg.rf.427270857e1afc8bd95c9c5164d8560d.jpg...
Predict count 1: 19 
 Predict count 2: 22
Processing /mnt/sda1/PythonProject/Pig_counting/dataset/mini_pig_200img/2023-04-12-09-50-03_2023-04-12-10-05-03_16486_jpg.rf.0c0d05839527eebc8c9ab776dc46ba7b.jpg...
Predict count 1: 57 
 Predict count 2: 49
Processing /mnt/sda1/PythonProject/Pig_counting/dataset/mini_pig_200img/1158_jpg.rf.593d549393ffde5c3d740cd491a618da.jpg...
Predict count 1: 46 
 Predict count 2: 46
Processing /mnt/sda1/PythonProject/Pig_counting/dataset/mini_pig_200img/2023-04-12-09-50-

In [8]:
from validation import process_validation_folder
from app import build_model_and_transforms, get_args_parser

# Setup model
parser = get_args_parser()
parser.set_defaults(pretrain_model_path="/mnt/sda1/PythonProject/Pig_counting/Pig_farming/train_countgd/checkpoints/papercheckpoint_thinh_train_199.pth")
args = parser.parse_args([])

model, transform = build_model_and_transforms(args)
model = model.to(get_device())

# Process validation dataset
results = process_validation_folder(
    model,
    transform,
    "/mnt/sda1/PythonProject/Pig_counting/dataset/PigsFarming_label_dataset_countgd/valid/images",  # Replace with your validation folder path
    "pig",                        # Text prompt for counting
    "/mnt/sda1/PythonProject/Pig_counting/Pig_farming/countgd/evaluate_papercheckpoint_thinh_train_199"       # Where to save results
)

Some weights of BertModel were not initialized from the model checkpoint at checkpoints/bert-base-uncased and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


final text_encoder_type: checkpoints/bert-base-uncased
load tokenizer done.
final text_encoder_type: checkpoints/bert-base-uncased
load tokenizer done.


TypeError: Object of type ndarray is not JSON serializable